In [ ]:
from chromalab.adaptation import dead_leaves

import matplotlib.colors as mcolors
import numpy as np
import polyscope as ps  # https://polyscope.run/py/
import random

In [ ]:
# Initialize polyscope and set some scene options
ps.init()

ps.set_build_gui(False)
ps.set_ground_plane_mode("none")
ps.set_SSAA_factor(4)
ps.set_background_color([0, 0, 0])

image_res = 512
ps.set_window_resizable(True)
ps.set_window_size(image_res / 2, image_res / 2) 

In [ ]:
n_channels = 3

if n_channels == 3:
    # Generate some RGB colors
    hues = np.linspace(0, 0.9, 100)
    color_palette = np.array([mcolors.hsv_to_rgb([hue, 1, 1]) for hue in hues])

    dead_leaves(0.5, color_palette, './out/dead_leaves_tri.png', shape_mode='mixed')

elif n_channels == 4:
    # Generate some tetra colors
    hues = np.linspace(0, 0.9, 100)
    color_palette = np.array([mcolors.hsv_to_rgb([hue, 1, 1]) for hue in hues])
    color_palette = np.hstack([color_palette, np.ones([color_palette.shape[0], 1])])

    dead_leaves(0.5, color_palette, './out/dead_leaves_tetra.png', shape_mode='mixed', tetra_save_mode='LMS_Q')